In [2]:
import numpy as np
import pandas as pd
comps = pd.read_csv("/kaggle/input/meta-kaggle/Competitions.csv")
evaluation = ['EvaluationAlgorithmAbbreviation',
              'EvaluationAlgorithmName',
              'EvaluationAlgorithmDescription',]
compt = ['Title', 'EnabledDate', 'HostSegmentTitle']
df = comps[compt + evaluation].copy()
df['year'] = pd.to_datetime(df.EnabledDate).dt.year.values
df['comps'] = 1
time_select = df.year >= 2015
competition_type_select = df.HostSegmentTitle.isin(['Featured', 'Research'])
pd.pivot_table(df[time_select&competition_type_select],
               values='comps',
               index=['EvaluationAlgorithmAbbreviation'],
               columns=['year'],
               fill_value=0.0,
               aggfunc=np.sum,
               margins=True).sort_values(by=('All'), ascending=False).iloc[1:,:].head(20)

year,2015,2016,2017,2018,2019,2020,2021,2022,2023,All
EvaluationAlgorithmAbbreviation,,,,,,,,,,
AUC,4,4,1,3,3,2,3,1,0,21
LogLoss,2,2,5,2,3,2,0,2,0,18
MAP@{K},1,3,0,4,1,0,2,3,0,14
FScoreBetaMicro,1,0,1,2,1,2,1,1,0,9
MulticlassLoss,2,3,2,0,1,0,0,1,0,9
CategorizationAccuracy,1,0,4,0,1,2,0,1,0,9
RMSLE,2,1,3,1,1,0,0,0,0,8
QuadraticWeightedKappa,3,0,0,1,2,1,0,0,0,7
RMSE,1,1,0,3,0,0,2,0,0,7


In [3]:
metric = 'AUC'
metric_select = df['EvaluationAlgorithmAbbreviation']==metric
print(df[time_select&competition_type_select&metric_select]
      [['Title', 'year']])

                                                  Title  year
235                          West Nile Virus Prediction  2015
261                       Springleaf Marketing Response  2015
263                                       Truly Native?  2015
280                           Homesite Quote Conversion  2015
311                     Santander Customer Satisfaction  2016
332                       Avito Duplicate Ads Detection  2016
346                   Predicting Red Hat Business Value  2016
368   Melbourne University AES/MathWorks/NIH Seizure...  2016
567       WSDM - KKBox's Music Recommendation Challenge  2017
782    TalkingData AdTracking Fraud Detection Challenge  2018
855                            Home Credit Default Risk  2018
1037          Santander Customer Transaction Prediction  2019
1078                       Microsoft Malware Prediction  2018
1665                              Instant Gratification  2019
1668                           IEEE-CIS Fraud Detection  2019
2495   J

**Handling never-before-seen metrics**

In [4]:
counts = (df[time_select&competition_type_select].groupby('EvaluationAlgorithmAbbreviation'))
total_comps_per_year = (df[time_select&competition_type_select].groupby('year').sum())
single_metrics_per_year = (counts.sum()[counts.sum().comps==1].groupby('year').sum())
single_metrics_per_year
table = (total_comps_per_year.rename(columns={'comps': 'n_comps'})
         .join(single_metrics_per_year / total_comps_per_year)
         .rename(columns={'comps': 'pct_comps'}))
print(table)

      n_comps  pct_comps
year                    
2015       28   0.142857
2016       19   0.157895
2017       34   0.117647
2018       35   0.200000
2019       36   0.277778
2020       44   0.272727
2021       31   0.354839
2022       38   0.421053
2023       12   0.916667


/tmp/ipykernel_32/377066418.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_comps_per_year = (df[time_select&competition_type_select].groupby('year').sum())
/tmp/ipykernel_32/377066418.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  single_metrics_per_year = (counts.sum()[counts.sum().comps==1].groupby('year').sum())
/tmp/ipykernel_32/377066418.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  single_metr

In [5]:
print(counts.sum()[counts.sum().comps==1].index.values)

['34817366' '35896185' '36031993' '37085174' '39078087' 'AHD@{Type}'
 'AI4CodeKendallTau' 'AmexGiniAndPercentageCaptureX' 'BenetechMixedMatch'
 'CSIROObjectDetectionFBeta' 'CVPRAutoDrivingAveragePrecision'
 'CernWeightedAuc' 'DFLEventDetectionAP' 'Dice3DHausdorff' 'DiceFBeta'
 'FScore_1 (deprecated)' 'GroupMeanLogMAE' 'ImageMatchingChallengeMaa'
 'ImageNetObjectLocalization' 'IndoorLocalization'
 'IntersectionOverUnionObjectSegmentationBeta'
 'IntersectionOverUnionObjectSegmentationWithClassification'
 'IntersectionOverUnionObjectSegmentationWithF1' 'JPXSharpe'
 'JaccardDSTLParallel' 'JaccardFbeta' 'JaneStreetPnl'
 'JigsawAgreementWithAnnotators' 'JigsawBiasAUC' 'LaplaceLogLikelihood'
 'LevenshteinMean' 'Lyft3DObjectDetectionAP' 'M5_WRMSSE' 'MASpearmanR'
 'MCSpearmanR' 'MSE' 'MeanAngularError' 'MeanColumnwiseLogLoss'
 'MeanColumnwiseMAE' 'MeanCosineSimilarity' 'MeanPearson' 'MeanPearsonOld'
 'MedicalBoardFBeta' 'MulticlassLossOld' 'NDCG@{K}'
 'NFLHelmetIdentification' 'NQMicroF1' 'NWRM

/tmp/ipykernel_32/2100426740.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(counts.sum()[counts.sum().comps==1].index.values)
/tmp/ipykernel_32/2100426740.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(counts.sum()[counts.sum().comps==1].index.values)
